In [ ]:

import math
import random
import numpy as np

ANTS = 10
MAX_ITERATION = 200
EVAPORATION_RATE = 0.1
EXPLORATION_RATE = 0.7
ALPHA = 1.0
BETA = 2.0


def read_input_file(file_name):
    file = open(file_name, "r")
    lines = file.readlines()
    file.close()

    coordinates = []
    for line in lines:
        coordinates.append({"x": int(line.split(" ")[1]), "y": int(line.split(" ")[2])})

    return coordinates


def distance(a, b):
    return math.sqrt((a["x"] - b["x"]) ** 2 + (a["y"] - b["y"]) ** 2)


def roulette_wheel_selection(probabilities):
    r = random.random()
    cumulative_probability = 0

    for probability, node in probabilities:
        cumulative_probability += probability
        if r <= cumulative_probability:
            return node


def update_pheromones(pheromones, ant_solutions):
    number_of_total_nodes = len(pheromones)
    new_pheromones = pheromones.copy()

    for i in range(number_of_total_nodes):
        for j in range(number_of_total_nodes):
            new_pheromones[i][j] *= 1 - EVAPORATION_RATE

    for solution, solution_cost in ant_solutions:
        for i in range(number_of_total_nodes - 1):
            node, next_node = solution[i], solution[i + 1]
            new_pheromones[node][next_node] += 1.0 / solution_cost
            new_pheromones[next_node][node] += 1.0 / solution_cost

    return new_pheromones


def construct_solution(coordinates, pheromones):
    number_of_total_nodes = len(coordinates)
    visited_node = [False] * number_of_total_nodes
    visited_node[0] = True
    solution = [0]

    current_node = 0
    solution_cost = 0

    for _ in range(number_of_total_nodes - 1):
        unvisited_nodes = [
            i for i in range(number_of_total_nodes) if not visited_node[i]
        ]
        if random.random() < EXPLORATION_RATE:
            probabilities = [
                (
                    (pheromones[current_node][i] ** ALPHA)
                    * (
                        (1.0 / distance(coordinates[current_node], coordinates[i]))
                        ** BETA
                    ),
                    i,
                )
                for i in unvisited_nodes
            ]
            _, next_node = max(probabilities)
        else:
            total_probability = sum(
                (pheromones[current_node][i] ** ALPHA)
                * ((1.0 / distance(coordinates[current_node], coordinates[i])) ** BETA)
                for i in unvisited_nodes
            )
            probabilities = [
                (
                    (pheromones[current_node][i] ** ALPHA)
                    * (
                        (1.0 / distance(coordinates[current_node], coordinates[i]))
                        ** BETA
                    )
                    / total_probability,
                    i,
                )
                for i in unvisited_nodes
            ]
            next_node = roulette_wheel_selection(probabilities)

        solution_cost += distance(coordinates[current_node], coordinates[next_node])
        solution.append(next_node)
        visited_node[next_node] = True
        current_node = next_node

    last_node = solution[-1]
    solution_cost += distance(coordinates[last_node], coordinates[0])

    return solution, solution_cost


def ant_colony_system(coordinates):
    number_of_total_nodes = len(coordinates)

    pheromones = [[1.0] * number_of_total_nodes for _ in range(number_of_total_nodes)]
    best_solution = None
    best_solution_cost = float("inf")

    for _ in range(MAX_ITERATION):
        ant_solutions = []

        for _ in range(ANTS):
            solution, solution_cost = construct_solution(coordinates, pheromones)
            ant_solutions.append((solution, solution_cost))

            if solution_cost < best_solution_cost:
                best_solution = solution
                best_solution_cost = solution_cost

        pheromones = update_pheromones(pheromones, ant_solutions)

    return best_solution, best_solution_cost


coordinates = read_input_file("Teszt1.tsp")
best_solution, best_solution_cost = ant_colony_system(coordinates)

print(f"\n>> The TSP path using ACS: \n\n{[coordinates[i] for i in best_solution]}\n")
print(f">> The TSP path cost using ACS: {best_solution_cost}")